In [20]:
import keras
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import utilities as util
import pixel as pix
import tile
import numpy as np

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="1";

In [3]:
landsat, sentinel, dem, label = util.load_data()
num_classes = util.get_class_count()
tile_gen_cnn = tile.tile_gen(landsat, sentinel, dem, label, 64, num_classes)
tile_gen_small = tile.tile_gen(landsat, sentinel, dem, label, 10, num_classes)
tile_gen_single = tile.tile_gen(landsat, sentinel, dem, label, 1, num_classes)

In [4]:
pixels = util.read_txt(["train_px.txt", "val_px.txt", "test_px.txt"])

In [5]:
train_px, val_px, test_px = (pixels[0], pixels[1], pixels[2])

In [7]:
image_train = tile_gen_single.tile_generator(train_px, len(train_px))
image_val = tile_gen_single.tile_generator(val_px, len(val_px))
image_test = tile_gen_single.tile_generator(test_px, len(test_px))

image_sk_train, train_labels = next(image_train) 
image_sk_val, val_labels = next(image_val) 
image_sk_test, test_labels = next(image_test)

image_sk_train = image_sk_train.reshape(len(train_px), 10)
image_sk_val = image_sk_val.reshape(len(val_px), 10)
image_sk_test = image_sk_test.reshape(len(test_px), 10)

In [9]:
print(train_labels.shape)
print(val_labels.shape)
print(test_labels.shape)

(519793, 16)
(155938, 16)
(113613, 16)


In [50]:
cnn_large = keras.models.load_model('/deep_data/resnet_fixed.h5')
cnn_small = keras.models.load_model('/deep_data/small_cnn_fixed.h5')
nn_single = keras.models.load_model('/deep_data/nn_model_fixed.h5')

In [51]:
batch_size = 25

In [52]:
train_predictions_single = nn_single.predict_generator(generator = tile_gen_single.tile_generator(val_px, batch_size=batch_size, flat=True), steps=len(val_px) // batch_size, verbose=1)

6237/6237 [==============================] - 549s 88ms/step


In [53]:
train_predictions_small = cnn_small.predict_generator(generator = tile_gen_small.tile_generator(val_px, batch_size=batch_size), steps=len(val_px) // batch_size, verbose=1)

6237/6237 [==============================] - 561s 90ms/step


In [54]:
train_predictions_large = cnn_large.predict_generator(generator = tile_gen_cnn.tile_generator(val_px, batch_size=batch_size), steps=len(val_px) // batch_size, verbose=1)

6237/6237 [==============================] - 641s 103ms/step


In [55]:
print(len(train_predictions_single))
input_data = list()
for i in range(len(train_predictions_single)):
    input_data.append(np.concatenate((train_predictions_single[i], train_predictions_small[i], train_predictions_large[i])))

155925


In [56]:
from keras import models
from keras import layers

In [57]:
model = models.Sequential()
model.add(layers.Dense(128, activation = "relu", input_shape=(48,)))
model.add(layers.Dense(128, activation = "relu"))
model.add(layers.Dense(128, activation = "relu"))
model.add(layers.Dense(16, activation = "softmax"))

In [58]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.0001), loss = 'categorical_crossentropy', metrics=['accuracy'])

In [59]:
model.fit(np.asarray(input_data), val_labels[:len(input_data)], epochs = 20, batch_size = 25)

Epoch 1/20
155925/155925 [==============================] - 56s 360us/step - loss: 1.2935 - acc: 0.5842
Epoch 2/20
155925/155925 [==============================] - 36s 229us/step - loss: 1.2023 - acc: 0.5978
Epoch 3/20
155925/155925 [==============================] - 36s 230us/step - loss: 1.1923 - acc: 0.5992
Epoch 4/20
155925/155925 [==============================] - 35s 226us/step - loss: 1.1849 - acc: 0.6007
Epoch 5/20
155925/155925 [==============================] - 35s 227us/step - loss: 1.1789 - acc: 0.6014
Epoch 6/20
155925/155925 [==============================] - 36s 228us/step - loss: 1.1736 - acc: 0.6031
Epoch 7/20
155925/155925 [==============================] - 35s 227us/step - loss: 1.1697 - acc: 0.6032
Epoch 8/20
155925/155925 [==============================] - 36s 229us/step - loss: 1.1660 - acc: 0.6032
Epoch 9/20
155925/155925 [==============================] - 36s 228us/step - loss: 1.1627 - acc: 0.6050
Epoch 10/20
155925/155925 [==============================] - 36s

In [60]:
test_single = nn_single.predict_generator(generator = tile_gen_single.tile_generator(test_px, batch_size=batch_size, flat=True), steps=len(test_px) // batch_size, verbose=1)
test_small = cnn_small.predict_generator(generator = tile_gen_small.tile_generator(test_px, batch_size=batch_size), steps=len(test_px) // batch_size, verbose=1)
test_cnn = cnn_large.predict_generator(generator = tile_gen_cnn.tile_generator(test_px, batch_size=batch_size), steps=len(test_px) // batch_size, verbose=1)

4544/4544 [==============================] - 458s 101ms/step


In [61]:
result_single = nn_single.evaluate_generator(generator = tile_gen_single.tile_generator(test_px, batch_size=batch_size, flat=True), steps=len(test_px) // batch_size, verbose=1)
result_small = cnn_small.evaluate_generator(generator = tile_gen_small.tile_generator(test_px, batch_size=batch_size), steps=len(test_px) // batch_size, verbose=1)
result_cnn = cnn_large.evaluate_generator(generator = tile_gen_cnn.tile_generator(test_px, batch_size=batch_size), steps=len(test_px) // batch_size, verbose=1)

4544/4544 [==============================] - 468s 103ms/step


In [62]:
print(result_single)
print(result_small)
print(result_cnn)

[1.3583110762418995, 0.5438556336555105]
[1.4874448318847677, 0.5260827462221692]
[1.4589961525446302, 0.542112675734835]


In [63]:
print(len(test_cnn))
print(len(test_single))
print(len(test_small))

113600
113600
113600


In [64]:
test_input_data = list()
for i in range(len(test_cnn)):
    test_input_data.append(np.concatenate((test_single[i], test_small[i], test_cnn[i])))

In [65]:
model.evaluate(x=np.asarray(test_input_data), y=test_labels[:len(test_input_data)], batch_size=128)

113600/113600 [==============================] - 13s 110us/step


[1.1476774147194875, 0.6060299295774648]